In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.silver;


In [0]:
bronze_df = spark.read.table("workspace.bronze.population_raw")

In [0]:
from pyspark.sql.functions import col, trim
from pyspark.sql.types import IntegerType, LongType, DoubleType

silver_df = (
    bronze_df
    .withColumn("state_code", trim(col("state_code")))
    .withColumn("rank_2014", col("rank_2014").cast(IntegerType()))
    .withColumn("population_estimate_2014", col("population_estimate_2014").cast(LongType()))
    .withColumn("median_sales_price_2015", col("median_sales_price_2015").cast(DoubleType()))
)

# Remove invalid records
silver_df = silver_df.dropna(subset=["state_code", "population_estimate_2014"])


In [0]:
silver_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.silver.population_clean")


In [0]:
%sql
SELECT * 
FROM workspace.silver.population_clean
LIMIT 10;